In [4]:
import requests, time

def get_cve_info(cve_id):
    if cve_id == 'X' or cve_id == '' or cve_id == None or cve_id != cve_id: return ''
    
    base_url = "https://services.nvd.nist.gov/rest/json/cves/2.0?"
    url = f"{base_url}cveId={cve_id}"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print(f"Success to retrieve CVE information. Status Code: {response.status_code}, CVE ID : {cve_id}")
            cve_info = response.json()
            time.sleep(8)
            return cve_info
        while response.status_code != 200:
            response = requests.get(url)
            if response.status_code == 200:
                print(f"Success to retrieve CVE information. Status Code: {response.status_code}, CVE ID : {cve_id}")
                cve_info = response.json()
                time.sleep(8)
                return cve_info
            else:
                print(f"Failed to retrieve CVE information. Status Code: {response.status_code}, CVE ID : {cve_id}")
                time.sleep(8)
    except Exception as e:
        print(f"An error occurred: {e}")
        
def get_cvss_score(json_data):
    cve_details = json_data.get("vulnerabilities", {})[0]['cve']['metrics']
    
    cvss_score = []
    for key in list(cve_details.keys()):
        cvss_data_list = cve_details.get(key)
        for cvss_data in cvss_data_list:
            cvss_base = cvss_data.get('cvssData').get('baseScore')
            cvss_score.append(str(cvss_base))
    
    return '/'.join(cvss_score)

def update_cvss_data(row):
    cve_id = row['CVE']
    cve_info = get_cve_info(cve_id)
    
    if cve_info:
        return get_cvss_score(cve_info)
    

In [5]:
import csv
import pandas as pd

if __name__ == "__main__":
    csv = pd.read_csv('vulnerability_list.csv')
    csv['CVSS'] = csv.apply(update_cvss_data, axis=1)

Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2021-38759
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2021-38545
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2018-18068
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2018-19860
Failed to retrieve CVE information. Status Code: 503, CVE ID : CVE-2020-9069
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2020-9069
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2019-5303
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2019-5292
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2019-5264
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2017-8216
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2019-5043
Success to retrieve CVE information. Status Code: 200, CVE ID : CVE-2019-5040
Failed to retrieve CVE information. Status Code: 503, CVE ID 

In [6]:
csv

,분류,제품명,CVE,설명,비고,Reference,CVSS
0,Microcontroller,Raspberry Pi 3,CVE-2021-38759,NaN,NaN,VARIoT,9.8/10.0
1,Microcontroller,Raspberry Pi 3,CVE-2021-38545,NaN,NaN,VARIoT,5.9/4.3
2,Microcontroller,Raspberry Pi 3,CVE-2018-18068,NaN,NaN,VARIoT,9.8/10.0
3,Microcontroller,Raspberry Pi 3,CVE-2018-19860,NaN,NaN,VARIoT,8.8/5.8
4,Smartphone,Huawei Honor 10 Lite,CVE-2020-9069,NaN,NaN,VARIoT,6.5/3.3
...,...,...,...,...,...,...,...
343,4G LTE IoT,NaN,CVE-2022-29730,NaN,NaN,NaN,9.8/10.0
344,LoRaWAN,NaN,CVE-2023-26494,NaN,NaN,NaN,6.1/6.1
345,LoRaWAN,NaN,CVE-2022-34022,NaN,NaN,NaN,7.2
346,LoRaWAN,NaN,CVE-2022-34021,NaN,NaN,NaN,5.4


In [9]:
csv.to_csv('iot_CVSSscore.csv', encoding='CP949')